20-fold cross-validation on all embeddings for each task. 

In [4]:
import sys
import os
import pickle

import numpy as np
import pandas as pd
from scipy import stats
from sklearn import metrics, model_selection

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
from embeddings_reproduction import gpm
from embeddings_reproduction import gpk

In [5]:
# assert np.__version__ == '1.13.1'
# assert pd.__version__ == '0.20.3'

In [6]:
def select_X_and_Y(df, all_X, y_column):
    not_dropped = ~pd.isnull(df[y_column])
    not_dropped = pd.Series(not_dropped, index=df.index)
    Ys = df[not_dropped][y_column]
    Ys.index = df[not_dropped]['name'].index
    #print(Ys)
    Xs = all_X.loc[Ys.index]
    
    #print(Xs)
    return Xs, Ys

def score(Y, pred_Y, pred_var):
    r1 = stats.rankdata(Y)
    r2 = stats.rankdata(pred_Y)
    scores = {}
    scores['kendalltau'] = stats.kendalltau(r1, r2).correlation
    scores['R2'] = metrics.r2_score(Y, pred_Y)
    scores['SE'] = metrics.mean_squared_error(Y, pred_Y)
    scores['R'] = np.corrcoef(Y, pred_Y)[0, 1]
    log_ps = -0.5 * np.log(pred_var) - (pred_Y - Y)**2 / 2 / pred_var
    log_ps -= 0.5 * np.log(2 * np.pi)
    scores['log_loss'] = -np.sum(log_ps)
    return scores

def cross_validate(y_col, df, e_dir, fname):
    with open(fname, 'w') as f:
        f.write('task,embedding,kernel,R,R2,kendalltau,log_loss,SE\n')
    for embed in os.listdir(e_dir):
        if embed[0] != 'X':
            continue
        with open(e_dir + embed, 'rb') as f:
            e_X = pickle.load(f)
        if len(e_X) == 2:
            e_X = e_X[0]
        #print(e_X)
        #print(df)
        X, y = select_X_and_Y(df, e_X, y_col)
        X = X.values
        y = y.values
        # Cross-validation predictions
        kf = model_selection.KFold(n_splits=20, shuffle=True, random_state=10)
        y_actual = []
        mu_val = {'cubic':[], 'Matern5/2':[]}
        var_val = {'cubic':[], 'Matern5/2':[]}
        mu_test = {}
        var_test = {}
        for i_train, i_val in kf.split(X):
            X_ = X[i_train]
            y_ = y[i_train]
            X_val = X[i_val]
            y_val = y[i_val]
            y_actual.append(y_val)
            k = gpk.MaternKernel('5/2')
            kernel = 'Matern5/2'
            clf = gpm.GPRegressor(k, gueses=(10, 100))
            clf.fit(X_, y_)
            mu, var = clf.predict(X_val)
            mu_val[kernel].append(mu)
            var_val[kernel].append(np.diag(var))
        y_actual = np.concatenate(y_actual)
        mu_val['Matern5/2'] = np.concatenate(mu_val['Matern5/2'])
        var_val['Matern5/2'] = np.concatenate(var_val['Matern5/2'])
        kernels = ['Matern5/2']
        val_scores_dict = {k:score(y_actual, mu_val[k], var_val[k]) for k in kernels}                            
        # Write to file
        for kernel in kernels:
            with open(fname, 'a') as f:
                scores = val_scores_dict[kernel]
                f.write(','.join([y_col, embed, kernel, str(scores['R']),
                                 str(scores['R2']), str(scores['kendalltau']),
                                 str(scores['log_loss']), str(scores['SE'])]))
                f.write('\n')

## SP Secretion CUT ALIGNED 

In [7]:
y_col = 'cut_secreted'
dataset = '../inputs/cut_align_sp_secretion.txt'
e_dir = '../outputs/cut_align_sp_secretion_embeddings/'
fname = '../outputs/cv_cut_align_sp_secretion.txt'

df = pd.read_csv(dataset)
    
%time cross_validate(y_col, df[df.is_train], e_dir, fname)

CPU times: user 12min 9s, sys: 37min 55s, total: 50min 4s
Wall time: 5min


In [9]:
## SP+Cutinase Secretion 

In [10]:
y_col = 'cut_secreted'
dataset = '../inputs/sp_secretion.txt'
e_dir = '../outputs/sp_secretion_embeddings/'
fname = '../outputs/cv_sp_secretion.txt'

df = pd.read_csv(dataset)
    
%time cross_validate(y_col, df[df.is_train], e_dir, fname)

CPU times: user 41min 44s, sys: 1h 40min 25s, total: 2h 22min 9s
Wall time: 7min 6s
